In [1]:
BATCH_SIZE = 64
EDGE_CROP = 16
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# downsampling inside the network
NET_SCALING = None
# downsampling in preprocessing
IMG_SCALING_2 = (2,2)
IMG_SCALING_3 = (3,3)
# number of validation images to use
VALID_IMG_COUNT = 900
# maximum number of steps_per_epoch in training
MAX_TRAIN_STEPS = 10000
MAX_TRAIN_EPOCHS = 5
SHUFFLE_BUFFER = 100000

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.utils import multi_gpu_model


from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from models.resnet50_classif import get_resnet50_classif, get_resnet50_classif_v2
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard, Callback

#from tensorflow.python import keras as keras
#import tensorflow as tf
#import os


Using TensorFlow backend.


In [3]:
import tensorflow as tf
with tf.Session() as sess:
    devices = sess.list_devices()
for device in devices:
    print(device)
    
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 7342465364229796312)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 18189167094826377132)
Please install GPU version of TF


In [4]:
ship_dir = '../../data/airbus_ship_detection/'
train_image_dir = os.path.join(ship_dir, 'train')# Images for training
test_image_dir = os.path.join(ship_dir, 'test')# Images for testing
label_dir = os.path.join(ship_dir, 'train_ship_segmentations_v2.csv')# Images for testing
masks = pd.read_csv(label_dir, engine="python") # Markers for ships

In [16]:
def _parse_function(proto):
    # define your tfrecord again. Remember that you saved your image as a string.
    keys_to_features = {'image': tf.FixedLenFeature([], tf.string),
                        "label": tf.FixedLenFeature([], tf.int64)}
    
    # Load one example
    parsed_features = tf.parse_single_example(proto, keys_to_features)
    
    # Turn your saved image string into an array
    parsed_features['image'] = tf.decode_raw(
        parsed_features['image'], tf.float32)
    
    return parsed_features['image'], parsed_features["label"]

In [17]:
def create_dataset(filepath, batch_size=BATCH_SIZE):
    
    # This works with arrays as well
    dataset = tf.data.TFRecordDataset(filepath)
    # Maps the parser on every filepath in the array. You can set the number of parallel loaders here
    dataset = dataset.map(_parse_function, num_parallel_calls=8)
    
    # This dataset will go on forever
    dataset = dataset.repeat()
    
    # Set the number of datapoints you want to load and shuffle 
    dataset = dataset.shuffle(256)
    
    # Set the batchsize
    dataset = dataset.batch(batch_size)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    # Create your tf representation of the iterator
    image, label = iterator.get_next()
    
    # Bring your picture back in shape
    image = tf.reshape(image, [-1, 768, 768, 3])
    
    # Create a one hot array for your labels
    label = tf.one_hot(label, 1)
    
    return image, label

In [18]:
tf_records_path_train = os.path.join(ship_dir, "training_clf.tfrecords")
tf_records_path_val = os.path.join(ship_dir, "validation_clf.tfrecords")

In [19]:
image, label = create_dataset(tf_records_path_train, batch_size=BATCH_SIZE)

In [9]:
weights_path_clf_scal2 = "weights_models/{}_weights.best.hdf5".format('clf_Scal2_tf_learn')

In [27]:
from keras.applications import ResNet50
from keras import models
from keras import layers

In [28]:
model_base = ResNet50(include_top=False, weights=None, input_tensor=image, input_shape=[768, 768, 3])
x = layers.Flatten() (model_base)
x = layers.Dense(128, activation='relu') (x)
Outputs = layers.Dense(1, activation='sigmoid') (x)

model =  models.Model(inputs=[model_base], outputs=[Outputs])
model.summary()

AttributeError: 'Node' object has no attribute 'output_masks'

In [ ]:
model = get_resnet50_classif_v2(input_shape=[768, 768, 3], weights=None, tensor=image)
model.summary()

In [14]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard, Callback
weight_path = "weights_models/{0}_weights.hdf5".format('model_1_scal1')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min', save_weights_only=True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                   patience=2, verbose=1, mode='min',
                                   min_delta=0.001, cooldown=1, min_lr=1e-7)

early = EarlyStopping(monitor="val_loss", mode="min", verbose=2,
                      patience=2) # probably needs to be more patient, but kaggle time is limited

tensorboard =  TensorBoard(log_dir="../logs/log1", update_freq='batch')

callbacks_list = [checkpoint, reduceLROnPlat, early]

In [ ]:
parallel_model = multi_gpu_model(model, gpus=2)
parallel_model.compile(optimizer=Adam(), loss=binary_crossentropy, 
              metrics=["accuracy"],
              target_tensors=[label])

In [ ]:
loss_history = [parallel_model.fit_generator(epochs=MAX_TRAIN_EPOCHS,
                                 callbacks=callbacks_list)]